In [1]:
import gurobipy as gb
from gurobipy import *
import numpy as np

# Rummikub

Rummikub is a game that combines elements of classic card games like Rummy with the strategy of tile placement. The game is played with a set of 106 tiles, with numbers ranging from 1 to 13 in four different colors (red, blue, yellow, and black). Additionally, there are two joker tiles in the set. The game is typically played by 2 to 4 players and revolves around the strategic placement and manipulation of numbered tiles. The objective of Rummikub is to be the first player to empty your rack of tiles by forming sets and runs of matching numbers. Sets consist of three or four tiles of the same number but different colors. For example, you could have a set of 3s with one red, one blue, and one black. Runs are sequences of at least three consecutive numbers of the same color. For instance, you could have a run of 4, 5, 6 in blue. The game continues until one player goes out, at which point they gain opponents' tile values, while others receive penalties determined by the remaining tiles in their racks.

In this project, our primary objective is to address Rummikub challenges through the application of integer linear programming. Initially, we plan to focus on a two-player scenario, with the potential to expand to a four-player format if time permits. Also, if time allows, our ultimate goal is to develop an interactive online Rummikub board game, providing users with a platform for engaging gameplay.

## Set up
$2*4*13 + joker*2$

In [2]:
Deck = {"Black": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
                  "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
        "Red": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
                  "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
        "Blue": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
                  "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
        "Orange": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
                  "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
        "Joker": ["J1", "J2"]}

In [3]:
Penalty = {"1": 1, "2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9": 9, "10":10, "11":11, "12":12, 
           "13":13, "Joker": 30}

In [4]:
# Value = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13]
# Value_matrix = Value * 4

In [5]:
model = gb.Model("Rummikub")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30


In [6]:
# Deck_types = ["Black", "Black2", "Red1", "Red2", "Blue1", "Blue2", "Orange", "Orange2"]
# Joker_values = ["J1", "J2"]
Deck_types = ["Black", "Red", "Orange", "Blue"]
Deck_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Joker = ["Joker"]
Joker_values = ["J"]


In [7]:
# I = len(Value_matrix)
# J = 1173

#### All Possible Sets

In [8]:
# Adjusting the setup to consider only the deck color
Deck_colors = ["Black", "Red", "Blue", "Orange"]

# Generating all possible sets with three consecutive numbers
# and represent each card in the set using Deck_color and Deck_values
three_consecutive_set_no_joker = []

for value in Deck_values[:-2]:  # Iterate through the values, stopping two before the end
    for color in Deck_colors:
        # Create a set of three consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2)]
        three_consecutive_set_no_joker.append(consecutive_set)


In [9]:
# Adjusting the setup to include one Joker in each set
# The Joker can replace any one of the three cards in the set

three_consecutive_set_one_joker = []
# Iterate through each color and value, creating sets with one Joker
for color in Deck_colors:
    for value in Deck_values[:-2]:  # Iterate through the values, stopping two before the end
        # For each set of three consecutive values, create three sets, each with one Joker
        if value == 1:
            for i in range(3):
                consecutive_set_one_joker = [(color, value), (color, value + 1), (color, value + 2)]
                consecutive_set_one_joker[i] = "Joker"  # Replace one card with a Joker
                three_consecutive_set_one_joker.append(consecutive_set_one_joker)
        else:
            for i in range(2):
                consecutive_set_one_joker = [(color, value), (color, value + 1), (color, value + 2)]
                consecutive_set_one_joker[i] = "Joker"  # Replace one card with a Joker
                three_consecutive_set_one_joker.append(consecutive_set_one_joker)


In [10]:
# Adjusting the setup to include exactly two Jokers in each set
# Each set will now consist of one card from the deck and two Jokers

# Generating all possible sets with one card from the deck and two Jokers
three_consecutive_sets_with_two_jokers = []

for color in Deck_colors:
    for value in Deck_values:  # Iterate through all values
        # Create a set with one card from the deck and two Jokers
        set_with_two_jokers = [(color, value), "Joker", "Joker"]
        three_consecutive_sets_with_two_jokers.append(set_with_two_jokers)

In [11]:
# Generating sets with four consecutive numbers, same color, and without Joker

four_consecutive_sets_no_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
        four_consecutive_sets_no_joker.append(consecutive_set)

In [12]:
four_consecutive_sets_one_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        if value == 1:
            for i in range(4):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                consecutive_set[i] = "Joker"
                four_consecutive_sets_one_joker.append(consecutive_set)
        else:
            for i in range(3):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                consecutive_set[i] = "Joker"
                four_consecutive_sets_one_joker.append(consecutive_set)

In [13]:
four_consecutive_sets_two_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        if value == 1:
            for i in range(3):
                for j in range(i+1, 4):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    four_consecutive_sets_two_joker.append(consecutive_set)
        else:
            for i in range(2):
                for j in range(i+1, 3):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    four_consecutive_sets_two_joker.append(consecutive_set)

In [14]:
# Generating sets with five consecutive numbers, same color, and without Joker

five_consecutive_sets_no_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
        five_consecutive_sets_no_joker.append(consecutive_set)

In [15]:
# Generating sets with five consecutive numbers, same color, and 1 Joker

five_consecutive_sets_one_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        if value == 1:
            for i in range(5):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                consecutive_set[i] = "Joker"
                five_consecutive_sets_one_joker.append(consecutive_set)
        else:
            for i in range(4):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                consecutive_set[i] = "Joker"
                five_consecutive_sets_one_joker.append(consecutive_set)

In [16]:
# Generating sets with five consecutive numbers, same color, and 2 Joker

five_consecutive_sets_two_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        if value == 1:
            for i in range(4):
                for j in range(i+1, 5):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    five_consecutive_sets_two_joker.append(consecutive_set)
        else:
            for i in range(3):
                for j in range(i+1, 4):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    five_consecutive_sets_two_joker.append(consecutive_set)

In [17]:
# Generating sets of three cards, each from a different color, without Jokers, and all cards having the same value

three_cards_different_colors_same_value = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            for k in range(j + 1, len(Deck_colors)):
                # Create a set with one card from each of the three different colors, all with the same value
                set_of_three = [(Deck_colors[i], value), (Deck_colors[j], value), (Deck_colors[k], value)]
                three_cards_different_colors_same_value.append(set_of_three)

In [18]:
# Generating sets of three cards, each from a different color, 1 Jokers, and all cards having the same value

three_cards_different_colors_1_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            # Create a set with one card from each of the three different colors, all with the same value
            set_of_three = [(Deck_colors[i], value), (Deck_colors[j], value), "Joker"]
            three_cards_different_colors_1_joker.append(set_of_three)

In [19]:
four_cards_different_colors_same_value = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    set_of_four = [(Deck_colors[0], value), (Deck_colors[1], value), (Deck_colors[2], value), (Deck_colors[3], value)]
    four_cards_different_colors_same_value.append(set_of_four)

In [20]:
# Generating sets of four cards, each from a different color, 1 Jokers, and all cards having the same value

four_cards_different_colors_1_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            for k in range(j + 1, len(Deck_colors)):
                # Create a set with one card from each of the three different colors, all with the same value
                set_of_four = [(Deck_colors[i], value), (Deck_colors[j], value), (Deck_colors[k], value), "Joker"]
                four_cards_different_colors_1_joker.append(set_of_four)

In [21]:
# Generating sets of four cards, each from a different color, 2 Jokers, and all cards having the same value

four_cards_different_colors_2_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            # Create a set with one card from each of the three different colors, all with the same value
            set_of_four = [(Deck_colors[i], value), (Deck_colors[j], value), "Joker", "Joker"]
            four_cards_different_colors_2_joker.append(set_of_four)

In [22]:
# three_consecutives = three_consecutive_set_no_joker + three_consecutive_set_one_joker + three_consecutive_sets_with_two_jokers
# four_consecutives = four_consecutive_sets_no_joker + four_consecutive_sets_one_joker + four_consecutive_sets_two_joker
# five_consecutives = five_consecutive_sets_no_joker + five_consecutive_sets_one_joker + five_consecutive_sets_two_joker
# three_same = three_cards_different_colors_same_value + three_cards_different_colors_1_joker
# four_same = four_cards_different_colors_same_value + four_cards_different_colors_1_joker + four_cards_different_colors_2_joker
# all_sets = three_consecutives + four_consecutives + five_consecutives + three_same + four_same

In [23]:
joker_sets = three_consecutive_set_one_joker + three_consecutive_sets_with_two_jokers + four_consecutive_sets_one_joker + \
    four_consecutive_sets_two_joker + five_consecutive_sets_one_joker + five_consecutive_sets_two_joker + \
        three_cards_different_colors_1_joker + four_cards_different_colors_1_joker + four_cards_different_colors_2_joker
J_joker = len(joker_sets)
J_joker

988

In [24]:
regular_sets = three_consecutive_set_no_joker + four_consecutive_sets_no_joker + five_consecutive_sets_no_joker + \
    three_cards_different_colors_same_value + four_cards_different_colors_same_value
J_reg = len(regular_sets)
J_reg

185

In [25]:
# n = 0
# for dt in Deck_types:
#     for dv in Deck_values:
#         for j in range(J_reg):
#             if (dt, dv) in regular_sets[j]:
#                 print((dt, dv), regular_sets[j])
#                 n += 1
# n


## Decision variables

In [26]:
S_reg = model.addVars(Deck_types, Deck_values, J_reg, vtype = GRB.BINARY, 
                  name = ["tile "+ dt + " " + str(dv) + " is in set " + str(regular_sets[j]) 
                          for dt in Deck_types for dv in Deck_values for j in range(J_reg)])
S_jok = model.addVars(Joker, Joker_values, J_joker, vtype = GRB.BINARY, 
                      name = ["tile Joker " + dv + " is in set " + str(joker_sets[j])
                              for dt in Joker for dv in Joker_values for j in range(J_joker)])

In [27]:
T_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile " + dt + " " + str(dv) + " on the table" for dt in Deck_types for dv in Deck_values])
T_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker " + dv + "on the table" for dt in Joker for dv in Joker_values])

In [28]:
R_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                   name = ["times of tile " + dt + " " + str(dv) +" on your rack" for dt in Deck_types for dv in Deck_values])
R_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker" + dv +"on your rack" for dt in Joker for dv in Joker_values])

In [29]:
Y_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile "+ dt + " " + str(dv) +" can be placed from your rack onto the table" 
                          for dt in Deck_types for dv in Deck_values])
Y_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker " + dv +" can be placed from your rack onto the table" 
                          for dv in Joker_values])

In [30]:
X_reg = model.addVars(J_reg, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of set "+str(regular_sets[j])+" can be placed onto the table" for j in range(J_reg)])
X_jok = model.addVars(J_joker, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of set "+str(joker_sets[j])+" can be placed onto the table" for j in range(J_joker)])

In [31]:
# W = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
#                   name = ["times of set "+str(j+1)+" on the table" for j in range(J)])
# M = 40
# Z = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
#                   name = ["times of set "+str(j+1)+" occurs in the old and in the new solution" for j in range(J)])

## Objective function

In [32]:
# model.setObjective(quicksum(Value[i] * Y[i] for i in range(I)) + 1/M * quicksum(Z[j] for j in range(J)), GRB.MAXIMIZE)
exp1 = quicksum(Deck_values[value-1] * Y_reg[types, value] for types in Deck_types for value in Deck_values)
exp2 = quicksum(Y_jok) * 30
model.setObjective(exp1 + exp2, GRB.MAXIMIZE)

## Constraints

In [33]:
# how many times a tile is on the table
model.addConstrs(quicksum(S_reg[dt, dv, j] * X_reg[j] for j in range(J_reg)) == T_reg[dt, dv] + Y_reg[dt, dv]
                 for dt in Deck_types for dv in Deck_values)
model.addConstr(quicksum(quicksum(S_jok) * X_jok[j] for j in range(J_joker)) == quicksum(T_jok) + quicksum(Y_jok))
#model.addConstrs(quicksum(S[i, j]*X[j] for j in range(J)) == T[i] + Y[i] for i in range(I))
model.update()

In [34]:
# whether a tile is in set J
model.addConstrs(S_jok["Joker", "J", j] == 1 for j in range(J_joker))
for dt in Deck_types:
    for dv in Deck_values:
        for j in range(J_reg):
            if (dt, dv) in regular_sets[j]:
                model.addConstr(S_reg[dt, dv, j] == 1)
            else:
                model.addConstr(S_reg[dt, dv, j] == 0)

In [35]:
# number of times moved from rack to table <= number of times on the rack
model.addConstrs(Y_reg[dt, dv] <= R_reg[dt, dv] for dt in Deck_types for dv in Deck_values)
model.addConstrs(Y_jok["Joker", dv] <= R_jok["Joker", dv] for dv in Joker_values)

{'J': <gurobi.Constr *Awaiting Model Update*>}

In [36]:
# number of times a set in a solution <= number of times a set being placed on the table
# model.addConstrs(Z[j] <= X[j] for j in range(J))
# number of times a set in a solution <= number of times a set on the table
# model.addConstrs(Z[j] <= W[j] for j in range(J))

In [37]:
# regular_sets.index([('Red', 11), ('Red', 12), ('Red', 13)])

In [38]:
# regular_sets[41]

In [39]:
# assuming this is the last step before end of the game
# thus no more tile drawing 
# model.addConstr(S_reg["Black", 1, 188] == 1) # tiles in set
model.addConstr(T_reg["Black", 1] >= 1) # times of tile on the table
model.addConstr(Y_reg["Black", 1] <= 1) # times of tile can be placed from rack to table
model.addConstr(R_reg["Black", 1] == 1) # times of tile on your rack

# model.addConstr(S_reg["Black", 2, 188] == 1)
model.addConstr(T_reg["Black", 2] >= 1)
model.addConstr(Y_reg["Black", 2] == 1) # original one card on table, now another one on rack, == 1
model.addConstr(R_reg["Black", 2] == 2)

# model.addConstr(S_reg["Black", 3, 188] == 1)
model.addConstr(T_reg["Black", 3] >= 1)
model.addConstr(Y_reg["Black", 3] <= 1)
model.addConstr(R_reg["Black", 3] == 1)

# model.addConstr(S_reg["Black", 4, 188] == 1)
model.addConstr(T_reg["Black", 4] >= 1)
model.addConstr(Y_reg["Black", 4] <= 1)
model.addConstr(R_reg["Black", 4] == 1)

# model.addConstr(S_reg["Black", 10, 36] == 1)
model.addConstr(T_reg["Black", 10] >= 1)
model.addConstr(Y_reg["Black", 10] == 1) # original one card on table, now another one on rack, == 1
model.addConstr(R_reg["Black", 10] == 2)

# model.addConstr(S_reg["Black", 11, 36] == 1)
model.addConstr(T_reg["Black", 11] >= 1)
model.addConstr(Y_reg["Black", 11] <= 1)
model.addConstr(R_reg["Black", 11] == 1)

# model.addConstr(S_reg["Black", 12, 36] == 1)
model.addConstr(T_reg["Black", 12] >= 1)
model.addConstr(Y_reg["Black", 12] <= 1)
model.addConstr(R_reg["Black", 12] == 1)

# model.addConstr(S_reg["Orange", 6, 23] == 1)
model.addConstr(T_reg["Orange", 6] >= 1)
model.addConstr(Y_reg["Orange", 6] <= 1)
model.addConstr(R_reg["Orange", 6] == 1)

# model.addConstr(S_reg["Orange", 7, 23] == 1)
model.addConstr(T_reg["Orange", 7] >= 1)
model.addConstr(Y_reg["Orange", 7] <= 1)
model.addConstr(R_reg["Orange", 7] == 1)

# model.addConstr(S_reg["Orange", 8, 23] == 1)
model.addConstr(T_reg["Orange", 8] >= 1)
model.addConstr(Y_reg["Orange", 8] == 1) # original one card on table, now another one on rack, == 1
model.addConstr(R_reg["Orange", 8] == 2)

# model.addConstr(S_reg["Orange", 9, 519] == 1)
model.addConstr(T_reg["Orange", 9] >= 1)
model.addConstr(Y_reg["Orange", 9] <= 1)
model.addConstr(R_reg["Orange", 9] == 1)

# model.addConstr(S_reg["Orange", 10, 519] == 1)
model.addConstr(T_reg["Orange", 10] >= 1)
model.addConstr(Y_reg["Orange", 10] <= 1)
model.addConstr(R_reg["Orange", 10] == 1)

# model.addConstr(S_reg["Orange", 11, 519] == 1)
model.addConstr(T_reg["Orange", 11] >= 1)
model.addConstr(Y_reg["Orange", 11] <= 1)
model.addConstr(R_reg["Orange", 11] == 1)

# model.addConstr(S_reg["Orange", 12, 519] == 1)
model.addConstr(T_reg["Orange", 12] >= 1)
model.addConstr(Y_reg["Orange", 12] <= 1)
model.addConstr(R_reg["Orange", 12] == 1)

# model.addConstr(S_reg["Orange", 13, 519] == 1)
model.addConstr(T_reg["Orange", 13] >= 1)
model.addConstr(Y_reg["Orange", 13] <= 1)
model.addConstr(R_reg["Orange", 13] == 1)

# model.addConstr(S_reg["Blue", 7, 26] == 1)
model.addConstr(T_reg["Blue", 7] >= 1)
model.addConstr(Y_reg["Blue", 7] <= 1)
model.addConstr(R_reg["Blue", 7] == 1)

# model.addConstr(S_reg["Blue", 8, 26] == 1)
model.addConstr(T_reg["Blue", 8] >= 1)
model.addConstr(Y_reg["Blue", 8] <= 1)
model.addConstr(R_reg["Blue", 8] == 1)

# model.addConstr(S_reg["Blue", 9, 26] == 1)
model.addConstr(T_reg["Blue", 9] >= 1)
model.addConstr(Y_reg["Blue", 9] <= 1)
model.addConstr(R_reg["Blue", 9] == 1)

# model.addConstr(S_reg["Blue", 10, 38] == 1)
model.addConstr(T_reg["Blue", 10] >= 1)
model.addConstr(Y_reg["Blue", 10] <= 1)
model.addConstr(R_reg["Blue", 10] == 1)

# model.addConstr(S_reg["Blue", 11, 38] == 1)
model.addConstr(T_reg["Blue", 11] >= 1)
model.addConstr(Y_reg["Blue", 11] <= 1)
model.addConstr(R_reg["Blue", 11] == 1)

# model.addConstr(S_reg["Blue", 12, 38] == 1)
model.addConstr(T_reg["Blue", 12] >= 1)
model.addConstr(Y_reg["Blue", 12] <= 1)
model.addConstr(R_reg["Blue", 12] == 1)

# model.addConstr(S_reg["Red", 11, 41] == 1)
model.addConstr(T_reg["Red", 11] >= 1)
model.addConstr(Y_reg["Red", 11] <= 1)
model.addConstr(R_reg["Red", 11] == 1)

# model.addConstr(S_reg["Red", 12, 41] == 1)
model.addConstr(T_reg["Red", 12] >= 1)
model.addConstr(Y_reg["Red", 12] <= 1)
model.addConstr(R_reg["Red", 12] == 1)

# model.addConstr(S_reg["Red", 13, 41] == 1)
model.addConstr(T_reg["Red", 13] >= 1)
model.addConstr(Y_reg["Red", 13] <= 1)
model.addConstr(R_reg["Red", 13] == 1)

for i in [22, 27, 36, 39, 41, 44, 110]:
    model.addConstr(X_reg[i] <= 1)

In [40]:
model.addConstr(R_reg["Orange", 1] >= 1)
model.addConstr(Y_reg["Orange", 1] <= 2)

model.addConstr(R_reg["Orange", 3] >= 1)
model.addConstr(Y_reg["Orange", 3] <= 2)

model.addConstr(R_reg["Orange", 4] >= 1)
model.addConstr(Y_reg["Orange", 4] <= 2)

model.addConstr(R_reg["Orange", 8] == 2)
# model.addConstr(Y_reg["Orange", 8] == 1)

model.addConstr(R_reg["Blue", 2] >= 1)
model.addConstr(Y_reg["Blue", 2] <= 2)

model.addConstr(R_reg["Black", 2] == 2)
# model.addConstr(Y_reg["Black", 2] <= 1) # already set in the cell above

model.addConstr(R_reg["Black", 10] == 2)
# model.addConstr(Y_reg["Black", 10] <= 1)

model.addConstr(R_reg["Black", 13] >= 1)
model.addConstr(Y_reg["Black", 13] <= 2)

model.addConstr(R_jok["Joker", "J"] >= 1)
model.addConstr(Y_jok["Joker", "J"] <= 2)

<gurobi.Constr *Awaiting Model Update*>

## Optimize

In [41]:
# model.params.BestObjStop = 331

In [42]:
model.optimize()
model.status

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10755 rows, 11940 columns and 10808 nonzeros
Model fingerprint: 0x8819fb5e
Model has 53 quadratic constraints
Variable types: 0 continuous, 11940 integer (10608 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 10703 rows and 11654 columns
Presolve time: 0.14s
Presolved: 52 rows, 286 columns, 781 nonzeros
Variable types: 0 continuous, 286 integer (28 binary)
Found heuristic solution: objective 264.0000000
Found heuristic solution: objective 286.0000000

Root relaxation: objective 5.160000e+02, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |

2

In [43]:
model.ObjVal

516.0

In [44]:
# for v in model.getVars():
#     if v.x != 0:
#         print (v.varName, v.x)

In [46]:
for j in range(J_reg):
    # for dt in Deck_types:
        # for dv in Deck_values:
            if X_reg[j].x != 0:
                print(X_reg[j].varName, X_reg[j].x)

times of set [('Black', 1), ('Black', 2), ('Black', 3)] can be placed onto the table 2.0
times of set [('Black', 2), ('Black', 3), ('Black', 4)] can be placed onto the table 1.0
times of set [('Orange', 4), ('Orange', 5), ('Orange', 6)] can be placed onto the table 1.0
times of set [('Orange', 5), ('Orange', 6), ('Orange', 7)] can be placed onto the table 1.0
times of set [('Blue', 6), ('Blue', 7), ('Blue', 8)] can be placed onto the table 1.0
times of set [('Red', 7), ('Red', 8), ('Red', 9)] can be placed onto the table 1.0
times of set [('Blue', 7), ('Blue', 8), ('Blue', 9)] can be placed onto the table 1.0
times of set [('Orange', 7), ('Orange', 8), ('Orange', 9)] can be placed onto the table 1.0
times of set [('Orange', 8), ('Orange', 9), ('Orange', 10)] can be placed onto the table 1.0
times of set [('Blue', 9), ('Blue', 10), ('Blue', 11)] can be placed onto the table 1.0
times of set [('Black', 10), ('Black', 11), ('Black', 12)] can be placed onto the table 1.0
times of set [('Re

In [49]:
for j in range(J_joker):
    # for dt in Deck_types:
        # for dv in Deck_values:
            if X_jok[j].x != 0:
                print(X_jok[j].varName, X_jok[j].x)